In [9]:
import requests
from bs4 import BeautifulSoup




In [10]:
!pip install selenium bs4
# You also need Chrome + ChromeDriver installed


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    # Setup headless browser
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)  # Wait for JS to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    data = {"center_id": center_id}

    # 1. Adreça
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # 2. Titularitat (Entitat titular)
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 3. Disponibilitat ("8 mesos")
    disponibilitat_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = disponibilitat_span.text.strip() if disponibilitat_span else None

    return data

# Example usage
center_id = 9747
result = extract_center_data(center_id)
for k, v in result.items():
    print(f"{k}: {v}")


center_id: 9747
adreça: C. Nou, 12
titularitat: Fundació Antònia Roura Barbany
temps_mitja_espera: 1 any i 9 mesosinfo


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    # Setup headless browser
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)  # Wait for JS to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    data = {"center_id": center_id}

    # ✅ 1. Adreça (beginner-friendly version)
    address_sections = soup.find_all('div', class_='center-gallery-info__section')
    addresses = []
    for section in address_sections:
        lines = section.find_all('div', class_='ctti-body')
        address = ' '.join(line.get_text(strip=True) for line in lines)
        if address:
            addresses.append(address)
    data["adreça"] = addresses[0] if addresses else None

    # ✅ 2. Titularitat (Entitat titular)
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # ✅ 3. Disponibilitat (basic clean without regex)
    disponibilitat_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    if disponibilitat_span:
        raw_text = disponibilitat_span.text.strip()
        cleaned_text = raw_text.replace("info", "").strip()  # manually remove 'info'
        data["temps_mitja_espera"] = cleaned_text
    else:
        data["temps_mitja_espera"] = None

    return data

# Example usage
center_id = 9747
result = extract_center_data(center_id)

# Display results
for k, v in result.items():
    print(f"{k}: {v}")

# Optional: DataFrame view
df = pd.DataFrame([result])
print(df)



center_id: 9747
adreça: C. Nou, 12 08401 Granollers,Vallès Oriental
titularitat: Fundació Antònia Roura Barbany
temps_mitja_espera: 1 any i 9 mesos
   center_id                                       adreça  \
0       9747  C. Nou, 12 08401 Granollers,Vallès Oriental   

                      titularitat temps_mitja_espera  
0  Fundació Antònia Roura Barbany    1 any i 9 mesos  


In [6]:
df

,center_id,adreça,titularitat,temps_mitja_espera
0,9747,"C. Nou, 12 08401 Granollers,Vallès Oriental",Fundació Antònia Roura Barbany,1 any i 9 mesos


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)  # JS content load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    data = {"center_id": center_id}

    # 1. Nom
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # 2. Adreça
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # 3. Titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 4. Temps mitjà d’espera
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = espera_span.text.strip() if espera_span else None

    # 5. Capacitat
    def get_capacitat_value(label_text):
        label_div = soup.find("div", string=lambda s: s and label_text in s)
        if label_div:
            value_span = label_div.find_next("span", class_="ctti-text-neutral-80 ctti-w700")
            if value_span:
                return value_span.text.strip()
        return None

    data["places_publiques"] = get_capacitat_value("Places públiques")
    data["places_privades"] = get_capacitat_value("Places privades")
    data["places_totals"] = get_capacitat_value("Places totals")

    # 6. Image (base64 source)
    image_tag = soup.find("img", class_="ng-star-inserted")
    data["image_base64"] = image_tag["src"] if image_tag and image_tag["src"].startswith("data:image") else None

    return data

# Example usage
center_id = 9747
result = extract_center_data(center_id)
for key, value in result.items():
    if key == "image_base64" and value:
        print(f"{key}: [base64 image string, length={len(value)}]")
    else:
        print(f"{key}: {value}")


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)  # Wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Quick check if page is valid
    if not soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title"):
        driver.quit()
        return None

    data = {"center_id": center_id}

    # 1. Descripció del servei
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # 2. Nom
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # 3. Adreça (updated to combine multiple lines)
    address_section = soup.find("div", class_="center-gallery-info__section")
    if address_section:
        lines = address_section.find_all("div", class_="ctti-body")
        full_address = ' '.join(line.get_text(strip=True) for line in lines)
        data["adreça"] = full_address
    else:
        data["adreça"] = None

    # 4. Titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 5. Temps mitjà d’espera (cleaned)
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    if espera_span:
        raw_text = espera_span.text.strip()
        cleaned_text = raw_text.replace("info", "").strip()
        data["temps_mitja_espera"] = cleaned_text
    else:
        data["temps_mitja_espera"] = None

    # 6. Places públiques, privades, totals
    capacitat = {}
    for div in soup.find_all("div", class_="ctti-body ng-star-inserted"):
        text = div.get_text(separator=" ", strip=True)
        if text.startswith("Places públiques:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_publiques"] = span.text.strip() if span else None
        elif text.startswith("Places privades:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_privades"] = span.text.strip() if span else None
        elif text.startswith("Places totals:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_totals"] = span.text.strip() if span else None
    data.update(capacitat)

    # 7. (Optional) Image skipped
    driver.quit()
    return data


def scrape_centers(start_id, end_id):
    all_data = []
    for center_id in range(start_id, end_id + 1):
        print(f"Scraping center ID: {center_id}")
        try:
            data = extract_center_data(center_id)
            if data:
                all_data.append(data)
            else:
                print(f"No data found for center ID {center_id}, skipping.")
        except Exception as e:
            print(f"Error scraping center ID {center_id}: {e}")

    return all_data


# 🔧 Set your ID range
start_id = 9353
end_id = 10484

data_list = scrape_centers(start_id, end_id)

# 💾 Save to CSV
df = pd.DataFrame(data_list)
df.to_csv("centersok.csv", index=False, encoding="utf-8-sig")

print(f"Scraping complete. Saved {len(data_list)} records to centersok.csv")


Scraping center ID: 9353
Scraping center ID: 9354
Scraping center ID: 9355
Scraping center ID: 9356
No data found for center ID 9356, skipping.
Scraping center ID: 9357
Scraping center ID: 9358
Scraping center ID: 9359
Scraping center ID: 9360
Scraping center ID: 9361
Scraping center ID: 9362
Scraping center ID: 9363
Scraping center ID: 9364
Scraping center ID: 9365
Scraping center ID: 9366
Scraping center ID: 9367
Scraping center ID: 9368
Scraping center ID: 9369
Scraping center ID: 9370
Scraping center ID: 9371
Scraping center ID: 9372
Scraping center ID: 9373
Scraping center ID: 9374
Scraping center ID: 9375
Scraping center ID: 9376
Scraping center ID: 9377
Scraping center ID: 9378
Scraping center ID: 9379
Scraping center ID: 9380
Scraping center ID: 9381
Scraping center ID: 9382
Scraping center ID: 9383
Scraping center ID: 9384
Scraping center ID: 9385
Scraping center ID: 9386
Scraping center ID: 9387
Scraping center ID: 9388
Scraping center ID: 9389
Scraping center ID: 9390
Scrapi

In [ ]:
df = pd.read_csv("centersok.csv", encoding="utf-8-sig")

# Show the first few rows
print(df.head())

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    data = {"center_id": center_id}

    # Descripció
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # Nom
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # Adreça
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # Titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # Temps mitjà d’espera
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = espera_span.text.strip() if espera_span else None

    # Capacitat - Places públiques, privades, totals
    labels = {
        "Places públiques": "places_publiques",
        "Places privades": "places_privades",
        "Places totals": "places_totals"
    }

    for label_text, field_name in labels.items():
        label_div = soup.find("div", string=lambda s: s and label_text in s)
        if label_div:
            value_span = label_div.find_next("span", class_="ctti-text-neutral-80 ctti-w700")
            if value_span:
                data[field_name] = value_span.text.strip()
            else:
                data[field_name] = None
        else:
            data[field_name] = None

    # Imatge base64
    image_tag = soup.find("img", class_="ng-star-inserted")
    data["image_base64"] = image_tag["src"] if image_tag and image_tag["src"].startswith("data:image") else None

    return data

# Example
center_id = 9752
result = extract_center_data(center_id)
for key, value in result.items():
    if key == "image_base64" and value:
        print(f"{key}: [image base64, length={len(value)}]")
    else:
        print(f"{key}: {value}")


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)  # wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    data = {"center_id": center_id}

    # Extract description
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # Extract name
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # Extract address
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # Extract titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

   
    # 4. Temps mitjà d’espera
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = espera_span.text.strip() if espera_span else None

    # Extract capacity numbers (places públiques, privades, totals)
    capacitat = {}
    for div in soup.find_all("div", class_="ctti-body ng-star-inserted"):
        text = div.get_text(separator=" ", strip=True)
        if text.startswith("Places públiques:"):
            capacitat["places_publiques"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places privades:"):
            capacitat["places_privades"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places totals:"):
            capacitat["places_totals"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
    data.update(capacitat)

    # Extract base64 image string
    image_tag = soup.find("img", class_="ng-star-inserted")
    data["image_base64"] = image_tag["src"] if image_tag and image_tag["src"].startswith("data:image") else None

    return data

# Example usage
center_id = 9482
result = extract_center_data(center_id)
for k, v in result.items():
    if k == "image_base64" and v:
        print(f"{k}: [image base64 string, length={len(v)}]")
    else:
        print(f"{k}: {v}")


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(10)  # wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Quick check if page is valid: for example, does it have the center name?
    if not soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title"):
        driver.quit()
        return None

    data = {"center_id": center_id}

    # Extract description
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # Extract name
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # Extract address
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # Extract titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 4. Temps mitjà d’espera
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = espera_span.text.strip() if espera_span else None

    # Extract capacity numbers (places públiques, privades, totals)
    capacitat = {}
    for div in soup.find_all("div", class_="ctti-body ng-star-inserted"):
        text = div.get_text(separator=" ", strip=True)
        if text.startswith("Places públiques:"):
            capacitat["places_publiques"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places privades:"):
            capacitat["places_privades"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places totals:"):
            capacitat["places_totals"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
    data.update(capacitat)

    # Extract base64 image string
    #image_tag = soup.find("img", class_="ng-star-inserted")
    #data["image_base64"] = image_tag["src"] if image_tag and image_tag["src"].startswith("data:image") else None

    driver.quit()
    return data


def scrape_centers(start_id, end_id):
    all_data = []
    for center_id in range(start_id, end_id + 1):
        print(f"Scraping center ID: {center_id}")
        try:
            data = extract_center_data(center_id)
            if data:
                all_data.append(data)
            else:
                print(f"No data found for center ID {center_id}, skipping.")
        except Exception as e:
            print(f"Error scraping center ID {center_id}: {e}")

    return all_data

# Parameters: change end_id as you want
start_id = 9408
end_id = 10300

data_list = scrape_centers(start_id, end_id)

# Save to CSV
df = pd.DataFrame(data_list)
df.to_csv("centers_data2.csv", index=False, encoding="utf-8-sig")

print(f"Scraping complete. Saved {len(data_list)} records to centers_data2.csv")


In [ ]:
#We tray again to scrape with the correct way of extracting the waiting time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)  # wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Quick check if page is valid: for example, does it have the center name?
    if not soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title"):
        driver.quit()
        return None

    data = {"center_id": center_id}

    # Extract description
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # Extract name
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # Extract address
    adreca_div = soup.find("div", class_="ctti-body")
    data["adreça"] = adreca_div.text.strip() if adreca_div else None

    # Extract titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 4. Temps mitjà d’espera
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    data["temps_mitja_espera"] = espera_span.text.strip() if espera_span else None

    # Extract capacity numbers (places públiques, privades, totals)
    capacitat = {}
    for div in soup.find_all("div", class_="ctti-body ng-star-inserted"):
        text = div.get_text(separator=" ", strip=True)
        if text.startswith("Places públiques:"):
            capacitat["places_publiques"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places privades:"):
            capacitat["places_privades"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
        elif text.startswith("Places totals:"):
            capacitat["places_totals"] = div.find("span", class_="ctti-text-neutral-80 ctti-w700").text.strip()
    data.update(capacitat)

    # Extract base64 image string
    #image_tag = soup.find("img", class_="ng-star-inserted")
    #data["image_base64"] = image_tag["src"] if image_tag and image_tag["src"].startswith("data:image") else None

    driver.quit()
    return data


def scrape_centers(start_id, end_id):
    all_data = []
    for center_id in range(start_id, end_id + 1):
        print(f"Scraping center ID: {center_id}")
        try:
            data = extract_center_data(center_id)
            if data:
                all_data.append(data)
            else:
                print(f"No data found for center ID {center_id}, skipping.")
        except Exception as e:
            print(f"Error scraping center ID {center_id}: {e}")

    return all_data

# Parameters: change end_id as you want
start_id = 9200
end_id = 9352

data_list = scrape_centers(start_id, end_id)

# Save to CSV
df = pd.DataFrame(data_list)
df.to_csv("centers_data1_00.csv", index=False, encoding="utf-8-sig")

print(f"Scraping complete. Saved {len(data_list)} records to centers_data1_00.csv")


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd

def extract_center_data(center_id):
    url = f"https://infocentres.esocial.gencat.cat/centres-extranet-front/cerca-de-centres/ambits/1/centre/{center_id}"

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(5)  # Wait for page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Quick check if page is valid
    if not soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title"):
        driver.quit()
        return None

    data = {"center_id": center_id}

    # 1. Descripció del servei
    desc_p = soup.find("p", class_="my-0 pt-2")
    data["descripcio_servei"] = desc_p.text.strip() if desc_p else None

    # 2. Nom
    name_h1 = soup.find("h1", class_="ctti-h2 ctti-text-neutral-60 app-h-card__title")
    data["nom"] = name_h1.text.strip() if name_h1 else None

    # 3. Adreça (updated to combine multiple lines)
    address_section = soup.find("div", class_="center-gallery-info__section")
    if address_section:
        lines = address_section.find_all("div", class_="ctti-body")
        full_address = ' '.join(line.get_text(strip=True) for line in lines)
        data["adreça"] = full_address
    else:
        data["adreça"] = None

    # 4. Titularitat
    titular_span = soup.find("span", class_="ctti-text-neutral-80 ctti-w700")
    data["titularitat"] = titular_span.text.strip() if titular_span else None

    # 5. Temps mitjà d’espera (cleaned)
    espera_span = soup.find("span", class_="fw-bold ctti-text-neutral-80 ng-star-inserted")
    if espera_span:
        raw_text = espera_span.text.strip()
        cleaned_text = raw_text.replace("info", "").strip()
        data["temps_mitja_espera"] = cleaned_text
    else:
        data["temps_mitja_espera"] = None

    # 6. Places públiques, privades, totals
    capacitat = {}
    for div in soup.find_all("div", class_="ctti-body ng-star-inserted"):
        text = div.get_text(separator=" ", strip=True)
        if text.startswith("Places públiques:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_publiques"] = span.text.strip() if span else None
        elif text.startswith("Places privades:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_privades"] = span.text.strip() if span else None
        elif text.startswith("Places totals:"):
            span = div.find("span", class_="ctti-text-neutral-80 ctti-w700")
            capacitat["places_totals"] = span.text.strip() if span else None
    data.update(capacitat)

    # 7. (Optional) Image skipped
    driver.quit()
    return data


def scrape_centers(start_id, end_id):
    all_data = []
    for center_id in range(start_id, end_id + 1):
        print(f"Scraping center ID: {center_id}")
        try:
            data = extract_center_data(center_id)
            if data:
                all_data.append(data)
            else:
                print(f"No data found for center ID {center_id}, skipping.")
        except Exception as e:
            print(f"Error scraping center ID {center_id}: {e}")

    return all_data


# 🔧 Set your ID range
start_id = 9353
end_id = 10484

data_list = scrape_centers(start_id, end_id)

# 💾 Save to CSV
df = pd.DataFrame(data_list)
df.to_csv("centersok.csv", index=False, encoding="utf-8-sig")

print(f"Scraping complete. Saved {len(data_list)} records to centersok.csv")
